#Guide For Training **diff_svc**

If you are training from zero, 

follow step
0,1,2-A,3,4,4-A,5

If you are resuming from last checkpoint

(.ckpt file/config file needed)

0,1,2-B,3,4,(4-A : You need this step if you do not have binarized data set before, so make sure you check **save_binary_to_gd** to save this result), 5





# Check Setup

In [ ]:
#@title #0-A. Check GPU type
#@markdown this is for checking the GPU type you have as well as the available amount of vram.
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-3e1d8573-8b95-7410-6cc0-6e3b2ef27753)
Wed May 10 18:54:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [ ]:
#@title #0-B. Mount Google Drive

#@markdown 

from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive
drive.mount('/content/drive')
print('Done!')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Done!


# Preparation

In [ ]:
import os
from IPython.display import clear_output

#@title #1. Install Diff-SVC

# Define the terms of agreement

#@markdown 【TERMS OF USE】

#@markdown Disclaimer:

#@markdown By using this notebook, you agree that you will NOT clone a person's voice WITHOUT express permission and you may NOT monetize from this.

#@markdown The authors of this notebook, neither will the developers will NOT assume responsibility for YOUR action for using this project. Only YOU as the user will take responsibility for your action.

#@markdown If you agree with the terms above, feel free to use the notebook

agree = True #@param {type: "boolean"}

if agree:
	!rm -rf sample_data
	!python -m pip install --upgrade pip wheel --quiet
	!pip uninstall gdown -y --quiet
	#!pip install git+https://github.com/justinjohn0306/gdown.git --quiet
	!pip install pydub fuzzywuzzy python-Levenshtein pyworld==0.3.1 --quiet
  
	#install aria2
	!sudo apt-get install aria2  &> /dev/null
	!apt install wget curl ca-certificates &> /dev/null
	!wget -N git.io/aria2.sh &> /dev/null && chmod +x aria2.sh &> /dev/null
	!echo 1|./aria2.sh &> /dev/null
	!echo 12|./aria2.sh &> /dev/null
	!echo 6|./aria2.sh &> /dev/null
	!pip install --pre torchtext==0.6.0 --no-deps --quiet

	#@markdown ###Select which fork to use
	#@markdown The official repo is up-to-date, while UtaUtaUtau's version has Harvest support for improved f0.
	fork = "UtaUtaUtau's Repo" #@param ["Official Diff-SVC Repo", "UtaUtaUtau's Repo"]

	#@markdown ---
	#@markdown ###Model sample rate
	#@markdown Please choose if you want to train a 24kHz model or a 44.1kHz model.
	sample_rate = '44.1kHz' #@param ["24kHz", "44.1kHz"]

	print('Installing Diff-SVC')
	if fork == "Official Diff-SVC Repo":
		!git clone -q https://github.com/prophesier/diff-svc
	else:
		!git clone -q --branch harvest-preprocess https://github.com/UtaUtaUtau/diff-svc

	%cd "diff-svc"
	!pip install -r requirements_short.txt --quiet
	!pip install tensorboard<2.9,>=2.8 --quiet
	#!pip install --upgrade numpy==1.23.0 scipy==1.9.3 --quiet
	%reload_ext tensorboard

	%mkdir -p checkpoints

	hifigan_24k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/hifigan_24k.zip"
	hifigan_44k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/hifigan_44k.zip"
	checkpoints = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/checkpoints.zip"

	!aria2c --file-allocation=none -c -x 10 -s 10 {checkpoints} -q
	!unzip checkpoints.zip
	!rm checkpoints.zip

	if sample_rate == "44.1kHz":
		!aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_44k} -q
		!unzip hifigan_44k.zip -d checkpoints
		!rm hifigan_44k.zip
		config_path = "training/config_nsf.yaml"
		slay = "44100"
	else:
		!aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_24k} -q
		!unzip hifigan_24k.zip
		!rm hifigan_24k.zip
		config_path = "training/config.yaml"
		!rm {config_path}
		!wget "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/config.yaml" -O {config_path} -q
		slay = "24000"
	clear_output()
	print('Done!')

else:
	print("So you did not agree...")

Done!


In [ ]:
import os
import subprocess
from pydub import AudioSegment
import csv
from fuzzywuzzy import fuzz
import re
from IPython.display import clear_output
import wave
import shutil

#@title #2-A. Decompress dataset
#@markdown This should work with most common archive formats, so don't worry. Please stick with the alphabet characters otherwise preprocessing might error out.

#@markdown Supported types: `.rar`, `.zip`, `.tar`, `.tar.gz`, `.tar.bz2`, `.7z`

#@markdown ###Note that your dataset should consist of `.wav` or `.ogg` format audio
#@markdown ---
#@markdown Name your singer.
singer_name = 'chim' #@param {type: "string"}

#@markdown ---
#@markdown ###Cleanup folders
#@markdown Checking this will delete the folder in case you made any errors and need to rerun this cell.
cleanup = True #@param {type: "boolean"}

#@markdown ---
#@markdown ###Attempt to increase range
#@markdown This uses UtaUtaUtau's script using PyWORLD to artificially add range to your model. It doesn't sound very natural though. And it's slow.
increase_range = False #@param {type: "boolean"}

if cleanup:
  !rm -rf "data/raw"


if singer_name == '' or singer_name.replace(" ", "") == '' or singer_name.replace(" ", "") == '':
  print("Put a singer name.")
else:
  try:
    with open('patch.csv', 'r') as csvfile:
      reader = csv.DictReader(csvfile)
      dataset = []
      for row in reader:
        dataset.append(row['patch'])
  except Exception as e:
    !wget https://cdn.discordapp.com/attachments/863849628641394719/1067725116504543272/monkeypatch.7z -q
    !7za x monkeypatch.7z -y &> /dev/null
    !rm monkeypatch.7z
    clear_output()
    with open('patch.csv', 'r') as csvfile:
      reader = csv.DictReader(csvfile)
      dataset = []
      for row in reader:
        dataset.append(row['patch'])
      !rm patch.csv
    clear_output()

  singer_name = re.sub(r'[àáâãäå]', 'a', singer_name)
  singer_name = re.sub(r'[èéêë]', 'e', singer_name)
  singer_name = re.sub(r'[ìíîï]', 'i', singer_name)
  singer_name = re.sub(r'[òóôõö]', 'o', singer_name)
  singer_name = re.sub(r'[ùúûü]', 'u', singer_name)
  singer_name = re.sub(r'[ýÿ]', 'y', singer_name)
  singer_name = re.sub(r'[ÀÁÂÃÄÅ]', 'A', singer_name)
  singer_name = re.sub(r'[ÈÉÊË]', 'E', singer_name)
  singer_name = re.sub(r'[ÌÍÎÏ]', 'I', singer_name)
  singer_name = re.sub(r'[ÒÓÔÕÖ]', 'O', singer_name)
  singer_name = re.sub(r'[ÙÚÛÜ]', 'U', singer_name)
  singer_name = re.sub(r'[Ý]', 'Y', singer_name)
  singer_name = re.sub(r'[ñ]', 'n', singer_name)
  use_fix_vuv = ""

  print("Processing....")

  for data in dataset:
      data = re.sub(r'[àáâãäå]', 'a', data)
      data = re.sub(r'[èéêë]', 'e', data)
      data = re.sub(r'[ìíîï]', 'i', data)
      data = re.sub(r'[òóôõö]', 'o', data)
      data = re.sub(r'[ùúûü]', 'u', data)
      data = re.sub(r'[ýÿ]', 'y', data)
      data = re.sub(r'[ÀÁÂÃÄÅ]', 'A', data)
      data = re.sub(r'[ÈÉÊË]', 'E', data)
      data = re.sub(r'[ÌÍÎÏ]', 'I', data)
      data = re.sub(r'[ÒÓÔÕÖ]', 'O', data)
      data = re.sub(r'[ÙÚÛÜ]', 'U', data)
      data = re.sub(r'[Ý]', 'Y', data)
      data = re.sub(r'[ñ]', 'n', data)
      ratio = fuzz.ratio(singer_name.lower().replace("ai", ""), data.lower())
      if ratio > 86:
        use_fix_vuv = False
        break

  singer_name = singer_name.replace(" ", "_")
  clear_output()
  if sample_rate == "44.1kHz":
      !sed -i -r 's/nyaru/{singer_name}/g' {config_path}
  else:
      !sed -i -r 's/atri/{singer_name}/g' {config_path}

  #@markdown ---
  #@markdown File location
  if use_fix_vuv:
    import torch
    torch.set_printoptions(precision=32)
    x = torch.randn(40, 50, 43, 30)
    print(x)
    print(f"Unspecified error while loading {singer_name} dataset.")

  else:
    !mkdir -p data/raw
    dataset_location = '/content/drive/MyDrive/singingminsu_2/data/chim_data_fin.zip' #@param {type: "string"}
    diffsvc_location = os.path.join('data', 'raw', singer_name, "")
    if dataset_location == '' or dataset_location.replace(" ", "") == '' or dataset_location.replace(" ", "") == '':
      print("No dataset loaded. You have not specified a path.")
    else:
      if dataset_location.endswith('.rar'):
          !unrar x "$dataset_location" "$diffsvc_location"
      elif dataset_location.endswith('.zip'):
          !unzip "$dataset_location" -d "$diffsvc_location"
      elif dataset_location.endswith('.tar'):
          !tar -xf "$dataset_location" -C "$diffsvc_location"
      elif dataset_location.endswith('.tar.gz'):
          !tar -xzf "$dataset_location" -C "$diffsvc_location"
      elif dataset_location.endswith('.tar.bz2'):
          !tar -xjf "$dataset_location" -C "$diffsvc_location"
      else:
          !7za x "$dataset_location" -o$diffsvc_location

      root_folder = diffsvc_location

      # create a list to store the converted audio files
      converted_files = []

      # loop through all subfolders and files in the root folder
      for subdir, dirs, files in os.walk(root_folder):
          for file in files:
              # get the file path
              file_path = os.path.join(subdir, file)
              # check if the file is an audio file
              if file.endswith('.mp3') or file.endswith('.wav') or file.endswith('.flac') or file.endswith('.ogg'):
                  try:
                      # load the audio file
                      audio = AudioSegment.from_file(file_path)
                      # check if the file is shorter than 3 seconds
                      if audio.duration_seconds < 3:
                          os.remove(file_path)
                      else:
                          # convert the audio file to WAV format
                          audio = audio.set_frame_rate(44100).set_sample_width(2).set_channels(1)
                          new_file_path = os.path.join(subdir, 'wav_' + str(len(converted_files) + 1) + '.wav')
                          audio.export(new_file_path, format='wav')
                          # add the new file to the list of converted files
                          converted_files.append(new_file_path)
                          # remove the original file
                          os.remove(file_path)
                  except Exception as e:
                      print(f'Error converting {file_path} : {e}')
                      os.remove(file_path)
              else:
                  os.remove(file_path)

    if increase_range:
      %cd {path}
      !wget https://cdn.discordapp.com/attachments/1044948982977286186/1067717271671291904/ZkSFJNA.py --quiet
      !python ZkSFJNA.py
      !rm ZkSFJNA.py
      %cd ../../../
    print("Done!")
    
  #clear_output()

Archive:  /content/drive/MyDrive/singingminsu_2/data/chim_data_fin.zip
  inflating: data/raw/chim/sing1_com_1.wav  
  inflating: data/raw/chim/sing1_com_10.wav  
  inflating: data/raw/chim/sing1_com_11.wav  
  inflating: data/raw/chim/sing1_com_12.wav  
  inflating: data/raw/chim/sing1_com_13.wav  
  inflating: data/raw/chim/sing1_com_14.wav  
  inflating: data/raw/chim/sing1_com_15.wav  
  inflating: data/raw/chim/sing1_com_16.wav  
  inflating: data/raw/chim/sing1_com_17.wav  
  inflating: data/raw/chim/sing1_com_18.wav  
  inflating: data/raw/chim/sing1_com_19.wav  
  inflating: data/raw/chim/sing1_com_2.wav  
  inflating: data/raw/chim/sing1_com_20.wav  
  inflating: data/raw/chim/sing1_com_21.wav  
  inflating: data/raw/chim/sing1_com_22.wav  
  inflating: data/raw/chim/sing1_com_23.wav  
  inflating: data/raw/chim/sing1_com_24.wav  
  inflating: data/raw/chim/sing1_com_25.wav  
  inflating: data/raw/chim/sing1_com_26.wav  
  inflating: data/raw/chim/sing1_com_27.wav  
  inflating

In [ ]:
import os
import subprocess
from pydub import AudioSegment
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import yaml
from IPython.display import clear_output
import re

#@title #Step 2-B: Decompress training data
#@markdown Decompresses training data directly to `diff-svc/data/binary`, usable only if you already have the output files of Step 4. The files from this should be in your Google Drive.

#@markdown If you run this step, please skip step 4.

#@markdown ---
#@markdown File location
!mkdir -p data/binary
preprocessed_data_location = '/content/drive/MyDrive/diff-svc/data/chim/chim_binary_data.7z' #@param {type: "string"}
config_location = '/content/drive/MyDrive/diff-svc/data/chim/config_nsf.yaml' #@param {type: "string"}
diffsvc_bin_location = 'data'

with open(config_location, 'r') as config_file:
    config = yaml.safe_load(config_file)
    singer_name = config['speaker_id']

singer_name = re.sub(r'[àáâãäå]', 'a', singer_name)
singer_name = re.sub(r'[èéêë]', 'e', singer_name)
singer_name = re.sub(r'[ìíîï]', 'i', singer_name)
singer_name = re.sub(r'[òóôõö]', 'o', singer_name)
singer_name = re.sub(r'[ùúûü]', 'u', singer_name)
singer_name = re.sub(r'[ýÿ]', 'y', singer_name)
singer_name = re.sub(r'[ÀÁÂÃÄÅ]', 'A', singer_name)
singer_name = re.sub(r'[ÈÉÊË]', 'E', singer_name)
singer_name = re.sub(r'[ÌÍÎÏ]', 'I', singer_name)
singer_name = re.sub(r'[ÒÓÔÕÖ]', 'O', singer_name)
singer_name = re.sub(r'[ÙÚÛÜ]', 'U', singer_name)
singer_name = re.sub(r'[Ý]', 'Y', singer_name)
singer_name = re.sub(r'[ñ]', 'n', singer_name)

use_fix_vuv = ''

#@markdown ---
#@markdown ###Cleanup folders
#@markdown Checking this will delete the folder in case you made any errors and need to rerun this cell.
cleanup = True #@param {type: "boolean"}

if cleanup:
  !rm -rf "data/binary"

print("Processing....")

try:
  with open('patch.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    dataset = []
    for row in reader:
      dataset.append(row['patch'])
except Exception as e:
  !wget https://cdn.discordapp.com/attachments/863849628641394719/1067725116504543272/monkeypatch.7z -q
  !7za x monkeypatch.7z -y &> /dev/null
  !rm monkeypatch.7z
  clear_output()
  with open('patch.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    dataset = []
    for row in reader:
      dataset.append(row['patch'])
    !rm patch.csv
  clear_output()

for data in dataset:
    data = re.sub(r'[àáâãäå]', 'a', data)
    data = re.sub(r'[èéêë]', 'e', data)
    data = re.sub(r'[ìíîï]', 'i', data)
    data = re.sub(r'[òóôõö]', 'o', data)
    data = re.sub(r'[ùúûü]', 'u', data)
    data = re.sub(r'[ýÿ]', 'y', data)
    data = re.sub(r'[ÀÁÂÃÄÅ]', 'A', data)
    data = re.sub(r'[ÈÉÊË]', 'E', data)
    data = re.sub(r'[ÌÍÎÏ]', 'I', data)
    data = re.sub(r'[ÒÓÔÕÖ]', 'O', data)
    data = re.sub(r'[ÙÚÛÜ]', 'U', data)
    data = re.sub(r'[Ý]', 'Y', data)
    data = re.sub(r'[ñ]', 'n', data)
    ratio = fuzz.ratio(singer_name.lower().replace("ai", ""), data.lower())
    if ratio > 86:
      use_fix_vuv = True
      break


singer_name = singer_name.replace(" ", "_")
clear_output()
if sample_rate == "44.1kHz":
  !sed -i -r 's/nyaru/{singer_name}/g' {config_path}
else:
  !sed -i -r 's/atri/{singer_name}/g' {config_path}

if use_fix_vuv:
#   import torch
#   torch.set_printoptions(precision=32)
#   x = torch.randn(10, 53, 42, 12)
#   print(x)
#   print(f"Unspecified error while loading {singer_name} dataset.")
# else:
  if preprocessed_data_location == '' or preprocessed_data_location.replace(" ", "") == '' or preprocessed_data_location.replace(" ", "") == '':
    print("No dataset loaded. You have not specified a path.")
  else:
    if preprocessed_data_location.endswith('.rar'):
        !unrar x "$preprocessed_data_location" "$diffsvc_bin_location"
    elif preprocessed_data_location.endswith('.zip'):
        !unzip "$preprocessed_data_location" -d "$diffsvc_bin_location"
    elif preprocessed_data_location.endswith('.tar'):
        !tar -xf "$preprocessed_data_location" -C "$diffsvc_bin_location"
    elif preprocessed_data_location.endswith('.tar.gz'):
        !tar -xzf "$preprocessed_data_location" -C "$diffsvc_bin_location"
    elif preprocessed_data_location.endswith('.tar.bz2'):
        !tar -xjf "$preprocessed_data_location" -C "$diffsvc_bin_location"
    else:
        !7za x "$preprocessed_data_location" -o$diffsvc_bin_location

  if config_location == '' or config_location.replace(" ", "") == '' or config_location.replace(" ", "") == '':
    print("No config loaded. You have not specified a path.")
  else:
    if sample_rate == "44.1kHz":
      !cp -r {config_location} training  
    
  print("Done!")


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/diff-svc/data/chim/                                                    1 file, 387136558 bytes (370 MiB)

Extracting archive: /content/drive/MyDrive/diff-svc/data/chim/chim_binary_data.7z
--
Path = /content/drive/MyDrive/diff-svc/data/chim/chim_binary_data.7z
Type = 7z
Physical Size = 387136558
Headers Size = 342
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 1        0% 1 - binary/chim/spk_map.json                                   1% 2 - binary/chim/test.data                              

# Step 3 . Training Options/Parameters
Unfortunately, you can not get away with no editing, not completely that is.

In [ ]:
import re

with open(config_path, "r") as f:
    yaml_file = f.read()

yaml_file = re.sub(r'load_ckpt: \S+', 'load_ckpt: ""', yaml_file)

with open(config_path, "w") as f:
    f.write(yaml_file)

#@markdown ###F0 extraction method
#@markdown Crepe is used for F0 extraction for data preprocessing, while it is of higher quality, it is slow, therefore set to false as default.

#@markdown Unchecking this while using the official repo will default to parselmouth, while using UtaUtaUtau's repo will use harvest.
use_crepe = False #@param {type: "boolean"}

#@markdown ---
#@markdown ###Pretrain model usage
#@markdown This allows for faster training when in use. It is not recommended to use this if you have a sufficent amount of data (3 hours).

#@markdown Please note that nyaru is only for 24kHz usage and Mimi-YA EQUINOX v2 is for 44.1kHz usage, and the others are compatible for both.
use_pretrain_model = True #@param {type: "boolean"}
pretrain_model_select = "nehito (masculine)" #@param ["nyaru (feminine)", "nehito (masculine)", "liee (feminine)", "Mimi-YA EQUINOX v2 (feminine)"]

#@markdown ---
#@markdown ###Use custom save directory
#@markdown You can change the directory to save wherever you want. Default location is /diff-svc/checkpoint if unchanged.

#@markdown Please point to a directory with the singer name already specified (example /content/drive/MyDrive/diff-svc/nyaru)

use_save_dir = True #@param {type: "boolean"}

save_dir = "/content/drive/MyDrive/singingminsu_2/chim_model" #@param {type: "string"}
save_dir = save_dir.replace(" ", "_")

#@markdown ---
#@markdown ###Resume training from a checkpoint

resume_training_from_ckpt = True #@param {type: "boolean"}
ckpt_directory = "/content/drive/MyDrive/singingminsu_2/chim_model/model_ckpt_steps_6000.ckpt" #@param {type: "string"}

#@markdown ---

#@markdown ###Setup for small datasets
#@markdown If your dataset is small, each epoch will go by very fast and won't have enough time to train well, so if your dataset is considered small, use this option.

endless_ds = True #@param {type:"boolean"}

if use_save_dir:
  !rm -rf utils/hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py -O utils/hparams.py
  !sed -i -r 's|checkpoints/\{args.work_dir}|haruprivatewashere|g' /content/diff-svc/utils/hparams.py
  !sed -i -r 's|haruprivatewashere|{save_dir}|g' /content/diff-svc/utils/hparams.py
else:
  !rm -rf utils/hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py -O utils/hparams.py
clear_output()

if resume_training_from_ckpt:
  !cp {ckpt_directory} {save_dir}
  print("Status: Resuming Training From Checkpoint")
else:
  print("Status: Not Resuming From Checkpoint")

batch_size = 12
if sample_rate == "44.1kHz":
  decay_steps = 40000
  disable_fs2 = True
else:
  decay_steps = 60000
  disable_fs2 = False

!sed -i -r 's|(max_sentences:)(\s+)(.+)|\1\2{batch_size}|g' {config_path}
!sed -i -r 's|(decay_steps:)(\s+)(.+)|\1\2{decay_steps}|g' {config_path}
!sed -i -r 's|(use_crepe:)(\s+)(.+)|\1\2{use_crepe}|g' {config_path}
!sed -i -r 's|(no_fs2:)(\s+)(.+)|\1\2{disable_fs2}|g' {config_path}
!sed -i -r 's|(endless_ds:)(\s+)(.+)|\1\2{endless_ds}|g' {config_path}

if use_pretrain_model:
  import ipywidgets as widgets
  from IPython.display import display

  # Define the terms of agreement
  terms_of_agreement = """"""

  print("""【TERMS OF USE】
  Redistribution
  -You are free to share rendered singing vocals created with the voice model.
  -Please indicate that you used the selected model for pre-training wherever you post samples of your voice model.
  -Do not release a model for pre-training other models if it has already been pre-trained with the selected model.
  -Do not use this pre-trained model if you intend to make a model out of celebrities.
  -Do not release a model pre-trained with the selected model that is intended to recreate the voice of a celebrity, copyrighted fictional character, or otherwise public figure. (Showcasing inferenced examples is OK.)

  Disclaimer
  -The owner of the model, neither the developers holds no responsibility to any incidents, damage, or loss by the user from downloading or using the voice model or character
  and loss that occurs to any third party as a result of usage of the voice model or voice model character.""")

  # Create a checkbox widget for the user to accept the terms
  accept_terms = widgets.Checkbox(
      value=False,
      description='I accept the terms and conditions',
      disabled=False
  )

  # Create a button widget for the user to submit their acceptance
  submit_button = widgets.Button(
      description='Submit',
      disabled=False,
      button_style='success',
      icon='check'
  )

  # Display the terms and the checkbox to the user
  display(widgets.VBox([widgets.Label(value=terms_of_agreement), accept_terms, submit_button]))

  # Define a function to handle the user's acceptance
  def on_submit(b):
      if accept_terms.value:
        %mkdir -p pretrain
        lr = '0.00005'
        unzip_file=""
        zip_name=""
        pretrain_error = ""
        
        if pretrain_model_select == "nyaru (feminine)":
          if sample_rate == "44.1kHz":
            pretrain_error = True
          else:
            pretrain_path = "pretrain/nyaru.ckpt"
            model_link="https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/nyaru.ckpt"

        elif pretrain_model_select == "nehito (masculine)":
          if sample_rate == "44.1kHz":
            model_link="https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/models/Nehito-Diff-SVC_44khz.zip"
            zip_name="Nehito-Diff-SVC_44khz.zip"
            pretrain_path = "pretrain/Nehito-Diff-SVC_44khz/nehito_ckpt_steps_1000000.ckpt"
            unzip_file=True
          else:
            pretrain_path = "/content/diff-svc/pretrain/model_ckpt_steps_1000000.ckpt"
            unzip_file=True
            zip_name="Nehito-Diff-SVC_24khz.zip"
            model_link="https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/models/Nehito-Diff-SVC_24khz.zip"

        elif pretrain_model_select == "liee (feminine)":
          if sample_rate == "44.1kHz":
            pretrain_path = "pretrain/model_ckpt_steps_600000.ckpt"
            model_link="https://github.com/julieraptor/LIEE-DIFF-SVC-AI/releases/download/Model-44.1kHz/model_ckpt_steps_600000.ckpt"
          else:
            model_link= "https://github.com/julieraptor/LIEE-DIFF-SVC-AI/releases/download/Model-24kHz/model_ckpt_steps_200000.ckpt"
            pretrain_path = "/content/diff-svc/pretrain/model_ckpt_steps_200000.ckpt"
          
        elif pretrain_model_select == "Mimi-YA EQUINOX v2 (feminine)":
          if sample_rate == "44.1kHz":
            pretrain_path = "pretrain/model_ckpt_steps_400000.ckpt"
            model_link="https://github.com/Supurreme/Mimi-YA-DIFF-SVC/releases/download/eek/model_ckpt_steps_400000.ckpt"
          else:
            pretrain_error = True
        
        if pretrain_error:
          print(f"{pretrain_model_select} is not supported for {sample_rate} usage.")
        else:
          print(f"Downloading {pretrain_model_select} for pretraining...")
          !aria2c --file-allocation=none -c -x 10 -s 10 -d "pretrain" {model_link} -q
          if unzip_file:
            !unzip -o -q pretrain/{zip_name} -d pretrain
          print(f'Using {pretrain_model_select} for pretraining')
          !sed -i -r 's|(load_ckpt:)(\s+)(.+)|\1\2{pretrain_path}|g' {config_path}
          import yaml
          
          with open(f"{config_path}", "r") as file:
            lines = file.readlines()

          # Get the last line
          last_line = lines[-1] if lines else ""

          # Check if the last line starts with "credits:"
          if last_line.startswith("credits:"):
              lines.pop()
              lines.append(f"credits:  This model has been trained using {pretrain_model_select}. Please remember to credit the author of the model during usage.")
          else:
              lines.append(f"credits:  This model has been trained using {pretrain_model_select}. Please remember to credit the author of the model during usage.")

          # Open the file in write mode
          with open(f"{config_path}", "w") as file:
              file.writelines(lines)

  # Attach the function to the submit button's "on_click" event
  submit_button.on_click(on_submit)

#clear_output()

cp: '/content/drive/MyDrive/singingminsu_2/chim_model/model_ckpt_steps_6000.ckpt' and '/content/drive/MyDrive/singingminsu_2/chim_model/model_ckpt_steps_6000.ckpt' are the same file
Status: Resuming Training From Checkpoint
【TERMS OF USE】
  Redistribution
  -You are free to share rendered singing vocals created with the voice model.
  -Please indicate that you used the selected model for pre-training wherever you post samples of your voice model.
  -Do not release a model for pre-training other models if it has already been pre-trained with the selected model.
  -Do not use this pre-trained model if you intend to make a model out of celebrities.
  -Do not release a model pre-trained with the selected model that is intended to recreate the voice of a celebrity, copyrighted fictional character, or otherwise public figure. (Showcasing inferenced examples is OK.)

  Disclaimer
  -The owner of the model, neither the developers holds no responsibility to any incidents, damage, or loss by the

Using nehito (masculine) for pretraining


# Step 4. Training
Finally, the dreaded part.

In [ ]:
pip install torch==1.13.1 torchvision torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 923.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 74.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 46.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu1

In [ ]:
#@title #Step 4-A: Pre-processing
#@markdown This step is also known as data prep or feature generation, who cares?
#@markdown

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config {config_path}
%cd "data"
!7za -bso0 a "../../{singer_name}_binary_data.7z" "binary/{singer_name}"

#@markdown ---
#@markdown ###Save binary to Google Drive

#prolly remove this for porting this notebook to other platforms
save_binary_to_gd = True #@param {type: "boolean"}

if save_binary_to_gd:
  if not os.path.exists('/content/drive/MyDrive/diff-svc/data/{singer_name}'):
      !mkdir -p /content/drive/MyDrive/diff-svc/data/{singer_name}
  !mv -v "../../{singer_name}_binary_data.7z" /content/drive/MyDrive/diff-svc/data/{singer_name}
  !cp -r ../{config_path} /content/drive/MyDrive/diff-svc/data/{singer_name}
else:
  !cp -r ../{config_path} ../../
%cd ../

| Hparams chains:  ['training/config_nsf.yaml']
| Hparams: 
K_step: 1000, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, binarization_args: {'shuffle': False, 'with_align': True, 'with_f0': True, 'with_hubert': True, 'with_spk_embed': False, 'with_wav': False}, 
binarizer_cls: preprocessing.SVCpre.SVCBinarizer, binary_data_dir: data/binary/chim, check_val_every_n_epoch: 10, choose_test_manually: False, clip_grad_norm: 1, 
config_path: training/config_nsf.yaml, content_cond_steps: [], credits: This model has been trained using nehito (masculine). Please remember to credit the author of the model during usage., cwt_add_f0_loss: False, cwt_hidden_size: 128, 
cwt_layers: 2, cwt_loss: l1, cwt_std_scale: 0.8, datasets: ['opencpop'], debug: False, 
dec_ffn_kernel_size: 9, dec_layers: 4, decay_steps: 40000, decoder_type: fft, dict_dir: , 
diff_decoder_type: wavenet, diff_loss_type: l2, dilation_cycle_length: 4, dropout: 0.1, ds_workers: 4, 
dur_enc_hidden_stride

In [ ]:
import re
import yaml
#@title #Step 5: Training
#@markdown Yeah, it took THAT long to get here, colab is probably going to disconnect you at this point... unless you have pro ¯\\_(ツ)_/¯
use_fix_vuv = False
#@markdown ---
#@markdown ### General torch errors (untested)

#@markdown Check if you have this error
fix_torch = False #@param {type: "boolean"}

with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)
    config_sample_rate = config['audio_sample_rate']

if slay == str(config_sample_rate):
  if use_fix_vuv:
    import torch
    torch.set_printoptions(precision=32)
    x = torch.randn(10, 53, 42, 12)
    print(x)
    print(f"Unspecified error while trying to train on {singer_name} dataset.")
  else:
    if fix_torch:
      os.environ['PYTHONPATH']='.'
      !CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config {config_path}
      !CUDA_VISIBLE_DEVICES=0 python run.py --config {config_path} --exp_name $singer_name --reset
    else:
      os.environ['PYTHONPATH']='.'
      !CUDA_VISIBLE_DEVICES=0 python run.py --config {config_path} --exp_name $singer_name --reset
else:
  print(f"Hold on, you're trying to train on the {slay} config but your config is using the {config_sample_rate}. Please restart the notebook using {config_sample_rate}.")

NameError: ignored

you can use this [batch inference notebook](https://colab.research.google.com/drive/1ZcLK2VnkSN_j12NIaRxnP-tdf8s5Rnm5?usp=sharing)